## Similarity to Features
This notebook explores the parameter sensitivity of similarity metrics

In [ ]:
import numpy as np
import os
from os.path import isfile, join
from os import listdir
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import csd_functions
import scipy
import reeb_matching
from copy import deepcopy
import scipy.spatial.distance as ssd
import scipy.cluster.hierarchy as hcluster
sns.set()

Load Similarity Matrices and compute dendrograms

In [ ]:
w_similarity_matrix = np.array(pd.read_csv('../data/gbarEvPyrAmpa_sweep/similarity_matrices/similarity_matrix_wasserstein.csv',header=None))
w_similarity_matrix = w_similarity_matrix/np.max(w_similarity_matrix)

#Make symmetric wrt upper triangle
i_lower = np.tril_indices(w_similarity_matrix.shape[0], -1)
w_similarity_matrix[i_lower] = w_similarity_matrix.T[i_lower]

%matplotlib inline
plt.figure(figsize=(5,5))
sns.set_style('darkgrid',{'axes.grid' : False})
plt.imshow(w_similarity_matrix)
plt.title('Topographic Similarity Matrix')
plt.colorbar()
plt.show()

In [ ]:
%matplotlib qt

distVec = ssd.squareform(w_similarity_matrix)
linkage = hcluster.linkage(distVec)
plt.figure()
dendro  = hcluster.dendrogram(linkage)

In [ ]:
%matplotlib inline
cluster_indeces = hcluster.fcluster(linkage, t=4, criterion='maxclust')
cluster_labels = np.unique(cluster_indeces)
p_dir = '../lfp_reeb_github/data/gbarEvPyrAmpa_sweep/points'
file_list = np.array(os.listdir(p_dir)) 

for label in cluster_labels:
    file_cluster = file_list[cluster_indeces == label]
    num_files = len(file_cluster)
    num_rc = np.ceil(np.sqrt(num_files)).astype(int)

    count = 1
    plt.figure()
    for f in file_cluster:
        f_path = p_dir + '/' + f

        plt.subplot(num_rc,num_rc,count)
        csd_grid = csd_functions.points2grid(np.array(pd.read_csv(f_path)))
        plt.imshow(csd_grid,aspect='auto')
        plt.axis('off')
        
        count += 1
    
    # plt.subplots_adjust(hspace=0.1, wspace=0.1)
    plt.subplots_adjust(hspace=0.1, wspace=0.1)
    plt.show()